# PART 1 - Creating Database

In [60]:
import numpy as np
import pandas as pd

In [61]:
url= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [62]:
treated = pd.DataFrame(url[0])

In [63]:
treated.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [64]:
treated = treated[treated["Borough"] != "Not assigned"]

In [65]:
treated.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [66]:
treated["Neighbourhood"][treated["Neighbourhood"] == "Not Assigned"] = treated["Borough"]

In [67]:
treated[0:20]

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [73]:
treated = pd.DataFrame(treated.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(lambda hoods: ", ".join(hoods)))

In [74]:
treated.reset_index(inplace=True)

In [75]:
treated.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [76]:
treated.shape

(103, 3)

# PART 2 - Adding Latitude and Longitude

In [87]:
import requests
import io

In [91]:
url2= "http://cocl.us/Geospatial_data"
geo=requests.get(url2).content
geodata=pd.read_csv(io.StringIO(geo.decode('utf-8')))

In [92]:
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [93]:
merged = pd.merge(geodata, treated, on="Postal Code")
merged= merged[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# PART 3 - Toronto Neighbourhood Analysis

I want to give a look into Central Toronto Region and it's Venues

In [110]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Libraries imported.


In [112]:
!conda install -c conda-forge folium=0.5.0 --yes

import folium

print('Folium installed and imported!')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.11.0             |   py37h89c1867_0         168 KB  conda-forge
    aiohttp-3.7.3              |   py37h5e8e339_2         629 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
   

libuuid-2.32.1       | 28 KB     | ##################################### | 100% 
appdirs-1.4.4        | 13 KB     | ##################################### | 100% 
jupyterlab-2.2.6     | 5.9 MB    | ##################################### | 100% 
qt-5.12.9            | 99.5 MB   | ##################################### | 100% 
async_generator-1.10 | 18 KB     | ##################################### | 100% 
tensorflow-estimator | 645 KB    | ##################################### | 100% 
nbconvert-6.0.7      | 535 KB    | ##################################### | 100% 
webencodings-0.5.1   | 12 KB     | ##################################### | 100% 
cached-property-1.5. | 10 KB     | ##################################### | 100% 
cachetools-4.2.1     | 13 KB     | ##################################### | 100% 
opt_einsum-3.3.0     | 51 KB     | ##################################### | 100% 
soupsieve-2.0.1      | 30 KB     | ##################################### | 100% 
astropy-4.2          | 7.5 M

tornado-6.1          | 646 KB    | ##################################### | 100% 
scipy-1.5.3          | 18.5 MB   | ##################################### | 100% 
libtiff-4.2.0        | 633 KB    | ##################################### | 100% 
blosc-1.21.0         | 841 KB    | ##################################### | 100% 
libblas-3.9.0        | 12 KB     | ##################################### | 100% 
glib-tools-2.66.7    | 85 KB     | ##################################### | 100% 
dask-core-2021.2.0   | 681 KB    | ##################################### | 100% 
nspr-4.29            | 232 KB    | ##################################### | 100% 
dill-0.3.3           | 60 KB     | ##################################### | 100% 
backports-1.0        | 4 KB      | ##################################### | 100% 
jedi-0.18.0          | 923 KB    | ##################################### | 100% 
imageio-2.9.0        | 3.1 MB    | ##################################### | 100% 
_libgcc_mutex-0.1    | 3 KB 

pyjwt-2.0.1          | 17 KB     | ##################################### | 100% 
pywavelets-1.1.1     | 4.4 MB    | ##################################### | 100% 
cx_oracle-8.1.0      | 193 KB    | ##################################### | 100% 
argon2-cffi-20.1.0   | 47 KB     | ##################################### | 100% 
ipython-7.20.0       | 1.1 MB    | ##################################### | 100% 
pluggy-0.13.1        | 29 KB     | ##################################### | 100% 
libxml2-2.9.10       | 1.3 MB    | ##################################### | 100% 
mock-4.0.3           | 51 KB     | ##################################### | 100% 
grpcio-1.35.0        | 2.1 MB    | ##################################### | 100% 
wcwidth-0.2.5        | 33 KB     | ##################################### | 100% 
libcurl-7.71.1       | 312 KB    | ##################################### | 100% 
pandas-1.2.2         | 11.7 MB   | ##################################### | 100% 
typing-extensions-3. | 8 KB 

done
Folium installed and imported!


Filtering only the Locations in Toronto

In [113]:
torontoonly = merged[merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
torontoonly.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Map of the Regions in Toronto

In [121]:
map_toronto = folium.Map(location=[torontoonly["Latitude"][0],torontoonly["Longitude"][0]],zoom_start=12)

for lat, lng, borough, neighborhood in zip(torontoonly['Latitude'], torontoonly['Longitude'], torontoonly['Borough'], torontoonly['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

As an exercise I'll focus on "Central Toronto" Region, which Latitude is 43.728020 and longitude -79.388790, data got from table above

In [151]:
central_to_lat = 43.7280
central_to_lon = -79.3887

In [152]:
central_to_lat

43.728

In [153]:
central_to_lon

-79.3887

Get Foursquare API Data on Central Toronto

In [132]:
CLIENT_ID = '30ICFWN2Y4NJH5QBO4FKQE2M3UG0BECYJJ4BQC5JDI3EZNHW' 
CLIENT_SECRET = 'XU10MFAQ1MXEPC1KUOL5MR4J2T3GHW0JNV4CZ4XA2OYEQAVP' 
ACCESS_TOKEN = '0L3U1BXFXGDPXEHGE2H3IPMGSMR3WONQMRPFDCOETU0S0MT5'
VERSION = '20180604'

In [154]:
LIMIT = 100
radius = 1000
url = ('https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            central_to_lat, 
            central_to_lon, 
            radius, 
            LIMIT))

results = requests.get(url).json()
        
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']
        
venues = results['response']['groups'][0]['items']

ce_to_venues = pd.json_normalize(venues)

ce_to_venues.head(5)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.postalCode
0,e-0-50e6da19e4b0d8a78a0e9794-0,0,"[{'summary': 'This spot is popular', 'type': '...",50e6da19e4b0d8a78a0e9794,Lawrence Park Ravine,3055 Yonge Street,Lawrence Avenue East,43.726963,-79.394382,"[{'label': 'display', 'lat': 43.72696303913755...",471,CA,Toronto,ON,Canada,"[3055 Yonge Street (Lawrence Avenue East), Tor...","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN
1,e-0-4b22e0a0f964a520544f24e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b22e0a0f964a520544f24e3,Granite Club,2350 Bayview Ave.,Lawrence Ave. West,43.733043,-79.381986,"[{'label': 'display', 'lat': 43.7330429446901,...",778,CA,Toronto,ON,Canada,"[2350 Bayview Ave. (Lawrence Ave. West), Toron...","[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",0,[],M2L 1E4
2,e-0-50a14c44e4b0bbf777f5152c-2,0,"[{'summary': 'This spot is popular', 'type': '...",50a14c44e4b0bbf777f5152c,Tim Hortons,2275 Bayview Ave,NaN,43.727324,-79.379563,"[{'label': 'display', 'lat': 43.72732441416303...",738,CA,Toronto,ON,Canada,"[2275 Bayview Ave, Toronto ON M4N 3M6, Canada]","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],M4N 3M6
3,e-0-50ed9da8e4b081eabee12672-3,0,"[{'summary': 'This spot is popular', 'type': '...",50ed9da8e4b081eabee12672,TTC Bus #162 - Lawrence-Donway,NaN,NaN,43.728026,-79.382805,"[{'label': 'display', 'lat': 43.72802605799448...",474,CA,Toronto,ON,Canada,"[Toronto ON, Canada]","[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",0,[],NaN
4,e-0-4ca34bc15720b1f77f5f30ef-4,0,"[{'summary': 'This spot is popular', 'type': '...",4ca34bc15720b1f77f5f30ef,Glendon Bookstore,2275 Bayview Ave,Bayview and Lawrence,43.727024,-79.378976,"[{'label': 'display', 'lat': 43.72702356473666...",789,CA,Toronto,ON,Canada,"[2275 Bayview Ave (Bayview and Lawrence), Toro...","[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",0,[],NaN


In [155]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
ce_to_venues = ce_to_venues.loc[:, filtered_columns]

ce_to_venues.columns = [col.split(".")[-1] for col in ce_to_venues.columns]

ce_to_venues.head(20)

,name,categories,lat,lng
0,Lawrence Park Ravine,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.726963,-79.394382
1,Granite Club,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",43.733043,-79.381986
2,Tim Hortons,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.727324,-79.379563
3,TTC Bus #162 - Lawrence-Donway,"[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",43.728026,-79.382805
4,Glendon Bookstore,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",43.727024,-79.378976
5,Glendon Forest,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",43.727226,-79.378413
6,Glendon Rose Garden,"[{'id': '4bf58dd8d48988d1aa941735', 'name': 'C...",43.727334,-79.378222
7,Lunik Co-op,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",43.727311,-79.377835
8,Glendon Athletic Club,"[{'id': '4bf58dd8d48988d1b2941735', 'name': 'C...",43.728799,-79.376819


In [162]:
map_ce_to = folium.Map(location=[ce_to_venues["lat"][0],ce_to_venues["lng"][0]],zoom_start=15)

for lat, lng, name in zip(ce_to_venues["lat"], ce_to_venues["lng"],ce_to_venues["name"]):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ce_to)  
    
map_ce_to